# Burnout en empleados corporativos
## Aprendizaje de Máquina - TP FINAL

---
## Desarrollo de Modelos de Ensamble
---

#### Carga de datos

In [35]:
#importar librerias
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler, label_binarize
from sklearn.metrics import classification_report, confusion_matrix, f1_score, roc_curve, roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import seaborn as sns
import numpy as np
from sklearn.ensemble import VotingClassifier


In [36]:
# Cargar de los datasets
X_train = pd.read_csv("output/burnout_X_train.csv")
y_train = pd.read_csv("output/burnout_y_train.csv")
X_test = pd.read_csv("output/burnout_X_test.csv")
y_test = pd.read_csv("output/burnout_y_test.csv")
X_train_PCA = pd.read_csv("output/burnout_X_train_PCA.csv").values
X_test_PCA = pd.read_csv("output/burnout_X_test_PCA.csv").values

In [37]:
#check unique values in y_train
print(y_train['BurnRate_Class'].unique())

[1. 0. 2.]


### Hard voting classifier

Best parameters found:  {'estimator__C': 100, 'estimator__max_iter': 100, 'estimator__solver': 'liblinear'}

kernel='rbf', C=1.0, gamma='scale', probability=True, random_state=42

K=13

In [38]:
lr1 = LogisticRegression(random_state=42, class_weight="balanced")
tree1 = DecisionTreeClassifier(max_depth=16, criterion='entropy', min_samples_split=17, min_samples_leaf=3, random_state=42)
svc_1 = SVC(C=5, kernel='linear', probability=True, random_state=42)
knn_1 = KNeighborsClassifier(n_neighbors=35, p=3, weights='distance')

estimators=[('Linear Regression', lr1), 
            ('Tree', tree1), 
            ('Support Vector Classifier', svc_1), 
            ('kNN', knn_1)] 



In [39]:
# Entrenar modelo 
hard_vot_clas = VotingClassifier(estimators=estimators, voting='hard', n_jobs=-1)
hard_vot_clas.fit(X_train, y_train)
# Predecir valores en el set de test
y_pred = hard_vot_clas.predict(X_test) 

report_original = classification_report(y_test, y_pred)
print("Reporte con datos originales:\n", report_original)

c:\Users\alope\AppData\Local\pypoetry\Cache\virtualenvs\ceia-ml-tpfinal--uYCcGwE-py3.11\Lib\site-packages\sklearn\preprocessing\_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\alope\AppData\Local\pypoetry\Cache\virtualenvs\ceia-ml-tpfinal--uYCcGwE-py3.11\Lib\site-packages\sklearn\preprocessing\_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


Reporte con datos originales:
               precision    recall  f1-score   support

         0.0       0.84      0.91      0.87      1136
         1.0       0.88      0.89      0.89      2514
         2.0       0.85      0.69      0.77       637

    accuracy                           0.87      4287
   macro avg       0.86      0.83      0.84      4287
weighted avg       0.87      0.87      0.87      4287



#### Utilizando datos transformados con PCA

In [40]:
# Entrenar modelo 
hard_vot_clas_PCA = VotingClassifier(estimators=estimators, voting='hard', n_jobs=-1)
hard_vot_clas_PCA.fit(X_train_PCA, y_train)
# Predecir valores en el set de test
y_pred_PCA = hard_vot_clas_PCA.predict(X_test_PCA) 

report_PCA = classification_report(y_test, y_pred_PCA)
print("Reporte con datos originales:\n", report_PCA)

c:\Users\alope\AppData\Local\pypoetry\Cache\virtualenvs\ceia-ml-tpfinal--uYCcGwE-py3.11\Lib\site-packages\sklearn\preprocessing\_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\alope\AppData\Local\pypoetry\Cache\virtualenvs\ceia-ml-tpfinal--uYCcGwE-py3.11\Lib\site-packages\sklearn\preprocessing\_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


Reporte con datos originales:
               precision    recall  f1-score   support

         0.0       0.82      0.91      0.86      1136
         1.0       0.87      0.86      0.87      2514
         2.0       0.78      0.65      0.71       637

    accuracy                           0.84      4287
   macro avg       0.82      0.81      0.81      4287
weighted avg       0.84      0.84      0.84      4287



Si observamos la performance de precision, recall y F1-score, observamos que en los tres casos disminuye ligeramente cuando utilizamos datos transformados con PCA, por lo que concluimos que es preferible utilizar los datos originales (recordando que los mismos ya fueron escalados y codificados). Cabe mencionar que al ser un hard voting classifier, no es posible medir el área bajo la curva ROC, al no predecir probabilidades.

### Soft voting classifer

Ahora entrenamos un Soft voting classifier, donde se promedia el output de probabilidad de cada modelo para obtener la probabilidad del ensamble, y utilizarla para hacer la predicción:

In [41]:
lr2 = LogisticRegression(random_state=42, class_weight="balanced")
tree2 = DecisionTreeClassifier(max_depth=16, criterion='entropy', min_samples_split=17, min_samples_leaf=3, random_state=42)
svc_2 = SVC(C=5, kernel='linear', probability=True, random_state=42)
knn_2 = KNeighborsClassifier(n_neighbors=35, p=3, weights='distance')

estimators_soft=[('Linear Regression', lr2), 
            ('Tree', tree2), 
            ('Support Vector Classifier', svc_2), 
            ('kNN', knn_2)] 



In [42]:
# Entrenar modelo 
soft_vot_clas = VotingClassifier(estimators=estimators, voting='soft', n_jobs=-1)
soft_vot_clas.fit(X_train, y_train)
# Predecir valores en el set de test
y_pred_soft = soft_vot_clas.predict(X_test) 

report_original_soft = classification_report(y_test, y_pred_soft)
print("Reporte con datos originales:\n", report_original_soft)

c:\Users\alope\AppData\Local\pypoetry\Cache\virtualenvs\ceia-ml-tpfinal--uYCcGwE-py3.11\Lib\site-packages\sklearn\preprocessing\_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\alope\AppData\Local\pypoetry\Cache\virtualenvs\ceia-ml-tpfinal--uYCcGwE-py3.11\Lib\site-packages\sklearn\preprocessing\_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


Reporte con datos originales:
               precision    recall  f1-score   support

         0.0       0.87      0.89      0.88      1136
         1.0       0.89      0.88      0.89      2514
         2.0       0.77      0.76      0.77       637

    accuracy                           0.87      4287
   macro avg       0.84      0.84      0.84      4287
weighted avg       0.87      0.87      0.87      4287



Con respecto al Hard-voting classifier, observamos que para las clases 0 y 1 en general no cambió el resultado. Sin embargo, para la clase 2, la más difícil de predecir por ser la minoritaria, empeoró el precision (pasó de 0.85 a 0.77), pero mejoró el recall (de 0.69 a 0.76), manteniendo el mismo F1-score. Esto implica que ahora detecta más casos de burnout severo (que es la clase 2), pero genera mayor cantidad de falsos positivos

In [43]:
# Entrenar modelo 
soft_vot_clas_PCA = VotingClassifier(estimators=estimators, voting='soft', n_jobs=-1)
soft_vot_clas_PCA.fit(X_train_PCA, y_train)
# Predecir valores en el set de test
y_pred_soft_PCA = soft_vot_clas_PCA.predict(X_test_PCA) 

report_PCA_soft = classification_report(y_test, y_pred_soft_PCA)
print("Reporte con datos originales:\n", report_PCA_soft)

c:\Users\alope\AppData\Local\pypoetry\Cache\virtualenvs\ceia-ml-tpfinal--uYCcGwE-py3.11\Lib\site-packages\sklearn\preprocessing\_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\alope\AppData\Local\pypoetry\Cache\virtualenvs\ceia-ml-tpfinal--uYCcGwE-py3.11\Lib\site-packages\sklearn\preprocessing\_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


Reporte con datos originales:
               precision    recall  f1-score   support

         0.0       0.84      0.87      0.85      1136
         1.0       0.87      0.85      0.86      2514
         2.0       0.72      0.70      0.71       637

    accuracy                           0.84      4287
   macro avg       0.81      0.81      0.81      4287
weighted avg       0.84      0.84      0.84      4287



### Ensamble de mejores modelos

A partir de los modelos base entrenados, utilizamos los mejores parámetros surgidos, y armamos los ensambles en base a los mismos:
- **LogisticRegression**: 'estimator__C': 100, 'estimator__max_iter': 100, 'estimator__solver': 'liblinear'
- **SVC**: kernel='rbf', C=1.0, gamma='scale'
- **KNeighborsClassifier**: n_neighbors=16
- **DecisionTreeClassifier**: utilizamos el modelo base

Se entrena únicamente el modelo en base a los datos originales, sin transformación de PCA, en base a los resultados obtenidos en los casos anteriores. En primer lugar entrenamos un Hard-voting classifier:

In [44]:
# Definir los estimadores con los parámetros optimizados
lr_optimo = LogisticRegression(C=100, max_iter=100, solver='liblinear', random_state=42, class_weight="balanced")
tree_optimo = DecisionTreeClassifier(max_depth=16, criterion='entropy', min_samples_split=17, min_samples_leaf=3, random_state=42)
svc_optimo = SVC(C=1.0, kernel='rbf', gamma='scale', probability=True, random_state=42)
knn_optimo = KNeighborsClassifier(n_neighbors=16, p=3, weights='distance')

# Lista de estimadores para el VotingClassifier
estimators_optimo = [
    ('Linear Regression', lr_optimo),
    ('Tree', tree_optimo),
    ('Support Vector Classifier', svc_optimo),
    ('kNN', knn_optimo)
]

# Configurar el VotingClassifier
hard_vot_clas_optimo = VotingClassifier(estimators=estimators_optimo, voting='hard', n_jobs=-1)
hard_vot_clas_optimo.fit(X_train, y_train)
# Predecir valores en el set de test
y_pred_optimo = hard_vot_clas_optimo.predict(X_test) 

report_original_optimo = classification_report(y_test, y_pred_optimo)
print("Reporte con datos originales:\n", report_original_optimo)

c:\Users\alope\AppData\Local\pypoetry\Cache\virtualenvs\ceia-ml-tpfinal--uYCcGwE-py3.11\Lib\site-packages\sklearn\preprocessing\_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\alope\AppData\Local\pypoetry\Cache\virtualenvs\ceia-ml-tpfinal--uYCcGwE-py3.11\Lib\site-packages\sklearn\preprocessing\_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


Reporte con datos originales:
               precision    recall  f1-score   support

         0.0       0.83      0.92      0.87      1136
         1.0       0.88      0.89      0.88      2514
         2.0       0.85      0.67      0.75       637

    accuracy                           0.86      4287
   macro avg       0.85      0.82      0.84      4287
weighted avg       0.86      0.86      0.86      4287



No se observa una mejoría respecto al Hard-voting classifier entrenado originalmente. Entrenamos ahora un Soft-voting classifier:

In [45]:
# Entrenar modelo 
soft_vot_clas_optimo = VotingClassifier(estimators=estimators, voting='soft', n_jobs=-1)
soft_vot_clas_optimo.fit(X_train, y_train)
# Predecir valores en el set de test
y_pred_soft_optimo = soft_vot_clas_optimo.predict(X_test) 

report_original_soft_optimo = classification_report(y_test, y_pred_soft_optimo)
print("Reporte con datos originales:\n", report_original_soft_optimo)

c:\Users\alope\AppData\Local\pypoetry\Cache\virtualenvs\ceia-ml-tpfinal--uYCcGwE-py3.11\Lib\site-packages\sklearn\preprocessing\_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\alope\AppData\Local\pypoetry\Cache\virtualenvs\ceia-ml-tpfinal--uYCcGwE-py3.11\Lib\site-packages\sklearn\preprocessing\_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


Reporte con datos originales:
               precision    recall  f1-score   support

         0.0       0.87      0.89      0.88      1136
         1.0       0.89      0.88      0.89      2514
         2.0       0.77      0.76      0.77       637

    accuracy                           0.87      4287
   macro avg       0.84      0.84      0.84      4287
weighted avg       0.87      0.87      0.87      4287



Nuevamente no se observa mejoría respecto al Soft-voting base. Esto posiblemente se de debido a la interacción entre los modelos, donde aunque cada modelo individual esté optimizado, la combinación de sus predicciones no lo sea si los mismos cometen errores similares, ya que estos modelos de ensamble requieren diversidad para mejorar la predicción